# Save Player Cumulative Season BoxScores 

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import time
from nba_api.stats.endpoints import leaguedashplayerstats, leaguegamelog
from tenacity import retry, stop_after_attempt, wait_fixed

pd.options.mode.chained_assignment =  None

box_DIR = "../../data/box/"
shiny_DIR = "../../data/shiny/"

In [5]:
season_start = 2024
season_end = 2025
seasons = np.arange(season_start,season_end,1).astype(str)

# Cumulative Boxscores

In [3]:
def update_box_base_p_w(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                league_id = "10",
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [4]:
update_box_base_p_w(seasons)

100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


In [13]:
dfa = []
for year in range(1997,season_end):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    with pd.option_context("future.no_silent_downcasting", True):
        df1 = df1.fillna(0).infer_objects(copy=False)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df["TS_PCT"] = round(df["PTS"]*0.5/((df["FGA"]+0.44*df["FTA"])),3)
df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.to_parquet(shiny_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [8]:
cols_i = ['GP', 'W', 'L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 
        'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT','OREB', 'DREB', 'REB', 
        'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS']

In [10]:
dfa = []
for year in tqdm(range(1997,season_end)):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    with pd.option_context("future.no_silent_downcasting", True):
        df1 = df1.fillna(0).infer_objects(copy=False)
    df1[df1["WL"] == 0]["WL"] = "L"
    df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_NAME"])
    keys = list(df1g.groups)
    dfb = []
    for key in keys:
        dfi = df1g.get_group(key)
        dfi["GP"] = 1
        dfi["W"] = np.where(dfi["WL"] == "W",1,0)
        dfi["L"] = np.where(dfi["WL"] == "L",1,0)
        dfi1 = dfi[cols_i]
        dfii = pd.DataFrame(dfi1.sum()).T
        dfii.iloc[:,3:] = dfii.iloc[:,3:]/dfii["GP"][0]
        dfii.iloc[:,3:] = dfii.iloc[:,3:].round(2)
        dfii["FG_PCT"] = round(dfii["FGM"]/dfii["FGA"],3)
        dfii["FG3_PCT"] = round(dfii["FG3M"]/dfii["FG3A"],3)
        dfii["FT_PCT"] = round(dfii["FTM"]/dfii["FTA"],3)
        dfii["TS_PCT"] = round(dfii["PTS"]*0.5/((dfii["FGA"]+0.44*dfii["FTA"])),3)
        dfii[["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_NAME"]] = list(key)
        dfb.append(dfii)
    dfb = pd.concat(dfb)
    dfb["season"] = year + 1
    dfa.append(dfb)
dfa = pd.concat(dfa)
dfs = dfa.replace([np.inf, -np.inf], np.nan)
dfs = dfs.fillna(0).reset_index(drop=True)
dfs.to_parquet(shiny_DIR+"WNBA_Box_P_Cum_Base_All.parquet")

100%|██████████| 28/28 [01:07<00:00,  2.40s/it]
